<a href="https://colab.research.google.com/github/yohanesnuwara/CoreImageDrivenML/blob/main/notebooks/2_Thin_Section_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will combine RCA and thin section image and calculate features for US Kansas and North Sea wells

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install mahotas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 37.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import mahotas as mt
from scipy.stats import skew, kurtosis

## Functions

In [4]:
def augment_image(image_path):
    """
    Perform augmentations (flipping and rotations) on the image.
    """
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        return None  # Return None if the image is not valid or doesn't exist

    # Perform augmentations
    augmented_images = {
        "original": image,
        "flipped_vertical": cv2.flip(image, 0),  # Vertical flip
        "flipped_horizontal": cv2.flip(image, 1),  # Horizontal flip
        "rotated_90": cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE),  # Rotate 90 degrees
        "rotated_180": cv2.rotate(image, cv2.ROTATE_180),  # Rotate 180 degrees
        "rotated_270": cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE),  # Rotate 270 degrees
    }
    return augmented_images

In [5]:
# Define the Haralick texture extraction function
def extract_texture(gray_clahe):
    # Calculate Haralick texture features for 4 types of adjacency
    textures = mt.features.haralick(gray_clahe)

    # Take the mean of it and return it
    ht_mean = textures.mean(axis=0)
    return ht_mean

# Add a column to store Haralick GLCM features
def process_image_and_extract_glcm(row):
    image_path = row['image_path']
    if os.path.exists(image_path):  # Check if the image file exists
        # Read the image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:  # Ensure the image was loaded successfully
            # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to the grayscale image
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            gray_clahe = clahe.apply(image)

            # Extract Haralick GLCM features
            return extract_texture(gray_clahe)
    return np.nan  # Return NaN if the image doesn't exist or cannot be processed

In [6]:
def extract_rgb_features(image_path):
    """Extract RGB features (mean, std, skewness) from non-black pixels of an image."""
    # Use OpenCV to read the image in color (BGR format)
    image = cv2.imread(image_path)  # Read the image in BGR color format
    if image is None:  # Check if the image is loaded properly
        return [np.nan] * 9  # Return NaNs if image cannot be read

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

    # Identify non-black pixels (non-transparent in original context)
    non_black_pixels = np.all(image_rgb != [0, 0, 0], axis=-1)

    # Extract the RGB values of non-black pixels
    r_values = image_rgb[:, :, 0][non_black_pixels]
    g_values = image_rgb[:, :, 1][non_black_pixels]
    b_values = image_rgb[:, :, 2][non_black_pixels]

    # Check if there are any non-black pixels
    if len(r_values) == 0:
        return [np.nan] * 9  # Return NaNs if there are no non-black pixels

    # Calculate mean, std, and skewness for non-black pixels
    r_mean, g_mean, b_mean = np.mean(r_values), np.mean(g_values), np.mean(b_values)
    r_std, g_std, b_std = np.std(r_values), np.std(g_values), np.std(b_values)
    r_skew, g_skew, b_skew = skew(r_values), skew(g_values), skew(b_values)

    # Return the RGB features as a flat list
    return [r_mean, g_mean, b_mean, r_std, g_std, b_std, r_skew, g_skew, b_skew]


def process_image_and_extract_features(row, index, total_rows):
    """Extract all features (GLCM and RGB statistics) for a single image."""
    if index % 100 == 0:  # Print progress every 100 rows
        print(f"Processing row {index}/{total_rows}...")

    image_path = row['image_path']
    if os.path.exists(image_path):  # Check if the image file exists
        # Read the image in grayscale for GLCM
        gray_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if gray_image is not None:
            # Apply CLAHE
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            gray_clahe = clahe.apply(gray_image)

            # Extract Haralick GLCM features
            haralick_features = extract_texture(gray_clahe)
        else:
            haralick_features = [np.nan] * 13  # 13 NaNs for GLCM features if image is invalid

        # Extract RGB features
        rgb_features = extract_rgb_features(image_path)

        # Combine features
        return list(haralick_features) + list(rgb_features)

    # Return NaNs if the file doesn't exist
    return [np.nan] * (13 + 9)  # 13 GLCM features + 9 RGB features

## 1 - Create dataframe for training data

In [7]:
# Filepath RCA
kgs_filepath = '/content/drive/MyDrive/Prores/ThinSection/RCA/Thin section analysis.xlsx'
norway_19a_filepath = '/content/drive/MyDrive/Prores/ThinSection/RCA/RCA_15_9_19A.xls'
norway_19bt2_filepath = '/content/drive/MyDrive/Prores/ThinSection/RCA/RCA_15_9_19BT2.xls'

# Load excel
kgs_df = pd.read_excel(kgs_filepath)

norway_19a_df = pd.read_excel(norway_19a_filepath, sheet_name='Selected', skiprows=1,
                   names=['core_no', 'plug_no', 'mMD', 'kgh', '1/pm_h', 'klh',
                          'kgv', '1/pm_v', 'klv', 'poroH', 'poroV', 'poro_sum',
                          'So', 'Sw', 'graindens_h', 'graindens_v', 'lith_desc',
                          'page_in_report', 'filename'])
norway_19a_df['well'] = '15-9-19 A'

norway_19bt2_df = pd.read_excel(norway_19bt2_filepath, sheet_name='Selected', skiprows=1,
                   names=['core_no', 'plug_no', 'mMD', 'kgh', '1/pm_h', 'klh',
                          'kgv', '1/pm_v', 'klv', 'poroH', 'poroV', 'poro_sum',
                          'So', 'Sw', 'graindens_h', 'graindens_v', 'lith_desc',
                          'page_in_report', 'filename'])
norway_19bt2_df['well'] = '15-9-19 BT2'


# Append both norway wells
norway_df = pd.concat([norway_19a_df, norway_19bt2_df], ignore_index=True)

kgs_df

,mMD,poro,k,m,n,qtz,dol,anhy,clay,potasfels,plagiofels,calci,pyr,hematite,well,Magnification_1,Magnification_2,Filename_Mag1,Filename_Mag2
0,2473.6,11.5,2.430,1.99,NaN,31.0,39.0,24.0,6.0,NaN,NaN,NaN,NaN,NaN,FLOWER A-1,40,160,2473.6_upper_edit.png,2473.6_lower_edit.png
1,2485.2,20.8,48.200,1.92,NaN,69.0,1.0,16.0,5.0,4.0,5.0,NaN,NaN,NaN,FLOWER A-1,40,160,2485.2_upper_edit.png,2485.2_lower_edit.png
2,2505.4,21.5,122.000,2.22,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLOWER A-1,40,160,2505.4_upper_edit.png,2505.4_lower_edit.png
3,2515.8,22.3,275.000,2.20,NaN,NaN,99.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,FLOWER A-1,40,160,2515.8_upper_edit.png,2515.8_lower_edit.png
4,2526.1,24.7,128.000,2.05,NaN,NaN,89.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,FLOWER A-1,40,160,2526.1_upper_edit.png,2526.1_lower_edit.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,2130.4,15.5,0.354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cyr A-1,32,128,2130.4_upper_edit.png,2130.4_lower_edit.png
85,2135.2,20.8,9.520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cyr A-1,32,128,2135.2_upper_edit.png,2135.2_lower_edit.png
86,2137.0,20.5,3.400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cyr A-1,32,128,2137_upper_edit.png,2137_lower_edit.png
87,2137.7,22.7,30.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cyr A-1,32,128,2137.7_upper_edit.png,2137.7_lower_edit.png


In [15]:
norway_df

,core_no,plug_no,mMD,kgh,1/pm_h,klh,kgv,1/pm_v,klv,poroH,poroV,poro_sum,So,Sw,graindens_h,graindens_v,lith_desc,page_in_report,filename,well
0,1,3,3837.55,25.2,0.746,21.4,3.94,0.495,3.16,10.8,,,,,2.69,,A.A.VW-cmt.w/o fis.Cl/Mic-lam.incr Dol.,3,page_3_upper.png,15-9-19 A
1,1,8,3838.50,1130,0.992,1080,1100,0.989,1040,17.2,,,,,2.66,,A.A.M-gr.,3,page_3_lower.png,15-9-19 A
2,1,13,3839.40,6.54,0.54,5.36,1200,0.99,1140,12.7,,,,,3.03,,A.A.w/o C-lam.,4,page_4_upper.png,15-9-19 A
3,1,18,3840.45,168,0.936,152,17.6,0.693,14.8,21,,,,,2.66,,A.A.,4,page_4_lower.png,15-9-19 A
4,1,23,3841.45,65,0.88,57,19.4,0.693,16.4,22.1,,,,,2.62,,A.A.F/M-gr.decr C.,5,page_5_upper.png,15-9-19 A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237,4,124,4103.75,45,0.83,39,,,,24.3,,,,,2.64,,Sst.Lt-gry.F-gr.Scat-M-gr.Sbang.W-cmt.W-srt.Lt...,34,page_34_upper.png,15-9-19 BT2
238,4,126,4104.75,35.6,0.786,30.6,,,,24.0,,,,,2.66,,a.a.Scs-C.,34,page_34_lower.png,15-9-19 BT2
239,4,128,4105.80,NMP,,NMP,,,,15.4,,,,,2.65,,Sst.Lt-gry.F/M-grSbang.W-cmt.W-srt.Frac.C/Cl/M...,35,page_35_upper.png,15-9-19 BT2
240,4,130,4106.45,28.1,0.766,24,,,,15.5,,,,,2.65,,"a.a.F/M-gr.Decr-Cl,Mic.w/o-C,Pyr.",35,page_35_lower.png,15-9-19 BT2


In [8]:
root_name = '/content/drive/MyDrive/Prores/ThinSection/Edited/'

Create image filepath inside dataframe

In [17]:
# Porosity dataframe combine KGS and north sea
porosity_kgs_df = kgs_df[['mMD', 'poro', 'k', 'well', 'Filename_Mag1']]
porosity_kgs_df = porosity_kgs_df.rename(columns={'poro': 'poroH', 'k': 'klh', 'Filename_Mag1': 'filename'})

porosity_norway_df = norway_df[['mMD', 'poroH', 'klh', 'graindens_h', 'well', 'filename']]

# Combine
thinsection_df = pd.concat([porosity_kgs_df, porosity_norway_df], ignore_index=True)

# Convert to float
thinsection_df['poroH'] = pd.to_numeric(thinsection_df['poroH'], errors='coerce')
thinsection_df['poroH'] = thinsection_df['poroH'].astype('float64')

thinsection_df['klh'] = pd.to_numeric(thinsection_df['klh'], errors='coerce')
thinsection_df['klh'] = thinsection_df['klh'].astype('float64')

# Generate thin section image path
thinsection_df['image_path'] = root_name + thinsection_df['well'] + '/' + thinsection_df['filename']

thinsection_df

,mMD,poroH,klh,well,filename,graindens_h,image_path
0,2473.60,11.5,2.43,FLOWER A-1,2473.6_upper_edit.png,NaN,/content/drive/MyDrive/Prores/ThinSection/Edit...
1,2485.20,20.8,48.20,FLOWER A-1,2485.2_upper_edit.png,NaN,/content/drive/MyDrive/Prores/ThinSection/Edit...
2,2505.40,21.5,122.00,FLOWER A-1,2505.4_upper_edit.png,NaN,/content/drive/MyDrive/Prores/ThinSection/Edit...
3,2515.80,22.3,275.00,FLOWER A-1,2515.8_upper_edit.png,NaN,/content/drive/MyDrive/Prores/ThinSection/Edit...
4,2526.10,24.7,128.00,FLOWER A-1,2526.1_upper_edit.png,NaN,/content/drive/MyDrive/Prores/ThinSection/Edit...
...,...,...,...,...,...,...,...
326,4103.75,24.3,39.00,15-9-19 BT2,page_34_upper.png,2.64,/content/drive/MyDrive/Prores/ThinSection/Edit...
327,4104.75,24.0,30.60,15-9-19 BT2,page_34_lower.png,2.66,/content/drive/MyDrive/Prores/ThinSection/Edit...
328,4105.80,15.4,NaN,15-9-19 BT2,page_35_upper.png,2.65,/content/drive/MyDrive/Prores/ThinSection/Edit...
329,4106.45,15.5,24.00,15-9-19 BT2,page_35_lower.png,2.65,/content/drive/MyDrive/Prores/ThinSection/Edit...


Check if every image exist in the dataframe

In [10]:
# Quick check if image exist
thinsection_df['Exist'] = thinsection_df['image_path'].apply(
    lambda x: os.path.exists(x) if isinstance(x, str) else False
)

thinsection_df

,mMD,poroH,klh,well,filename,image_path,Exist
0,2473.60,11.5,2.43,FLOWER A-1,2473.6_upper_edit.png,/content/drive/MyDrive/Prores/ThinSection/Edit...,True
1,2485.20,20.8,48.20,FLOWER A-1,2485.2_upper_edit.png,/content/drive/MyDrive/Prores/ThinSection/Edit...,True
2,2505.40,21.5,122.00,FLOWER A-1,2505.4_upper_edit.png,/content/drive/MyDrive/Prores/ThinSection/Edit...,True
3,2515.80,22.3,275.00,FLOWER A-1,2515.8_upper_edit.png,/content/drive/MyDrive/Prores/ThinSection/Edit...,True
4,2526.10,24.7,128.00,FLOWER A-1,2526.1_upper_edit.png,/content/drive/MyDrive/Prores/ThinSection/Edit...,True
...,...,...,...,...,...,...,...
326,4103.75,24.3,39.00,15-9-19 BT2,page_34_upper.png,/content/drive/MyDrive/Prores/ThinSection/Edit...,True
327,4104.75,24.0,30.60,15-9-19 BT2,page_34_lower.png,/content/drive/MyDrive/Prores/ThinSection/Edit...,True
328,4105.80,15.4,NaN,15-9-19 BT2,page_35_upper.png,/content/drive/MyDrive/Prores/ThinSection/Edit...,True
329,4106.45,15.5,24.00,15-9-19 BT2,page_35_lower.png,/content/drive/MyDrive/Prores/ThinSection/Edit...,True


## 2 - Replicate image (flipping & rotation)

In [ ]:
# Prepare a new DataFrame to store augmented data
augmented_rows = []

# Process each row in the DataFrame
for _, row in thinsection_df.iterrows():
    image_path = row['image_path']

    # Check if the image exists
    if os.path.exists(image_path):
        augmented_images = augment_image(image_path)

        if augmented_images is not None:
            for transform_name, transformed_image in augmented_images.items():
                # Save the augmented image to disk (optional)
                new_image_path = f"{os.path.splitext(image_path)[0]}_{transform_name}.png"
                cv2.imwrite(new_image_path, transformed_image)

                # Create a new row for the augmented image
                new_row = row.copy()  # Copy the original row
                new_row['image_path'] = new_image_path  # Update the image path
                new_row['augmentation'] = transform_name  # Add an augmentation type column
                augmented_rows.append(new_row)

# Add the augmented rows to the original DataFrame
augmented_df = pd.DataFrame(augmented_rows)
thinsection_df = pd.concat([thinsection_df, augmented_df], ignore_index=True)


In [ ]:
thinsection_df.to_csv('ThinSection_Augmented.csv', index=False)

## 3 - Extract GLCM

In [ ]:
# # Apply the function to extract Haralick GLCM features and store them in a new column
# thinsection_df['GLCM_features'] = thinsection_df.apply(process_image_and_extract_glcm, axis=1)

# # Optionally, split the GLCM features into separate columns
# glcm_columns = [f'GLCM_{i}' for i in range(13)]  # Haralick GLCM returns 13 features
# thinsection_df[glcm_columns] = pd.DataFrame(thinsection_df['GLCM_features'].tolist(), index=thinsection_df.index)

# # Drop the original GLCM_features column if splitting is done
# thinsection_df = thinsection_df.drop(columns=['GLCM_features'])

# # Display the updated DataFrame
# thinsection_df

In [19]:
# Process all rows
total_rows = len(thinsection_df)
all_features = []

for index, row in thinsection_df.iterrows():
    all_features.append(process_image_and_extract_features(row, index, total_rows))

# Add features to the DataFrame
feature_count = 13 + 9  # Haralick (13) + RGB (9)
feature_columns = [f'Feature_{i}' for i in range(feature_count)]
thinsection_df[feature_columns] = pd.DataFrame(all_features, index=thinsection_df.index)

Processing row 0/331...
Processing row 100/331...
Processing row 200/331...
Processing row 300/331...


In [ ]:
thinsection_df.columns

Index(['mMD', 'poroH', 'klh', 'well', 'filename', 'image_path', 'Exist',
       'GLCM_0', 'GLCM_1', 'GLCM_2', 'GLCM_3', 'GLCM_4', 'GLCM_5', 'GLCM_6',
       'GLCM_7', 'GLCM_8', 'GLCM_9', 'GLCM_10', 'GLCM_11', 'GLCM_12',
       'Feature_0', 'Feature_1', 'Feature_2', 'Feature_3', 'Feature_4',
       'Feature_5', 'Feature_6', 'Feature_7', 'Feature_8', 'Feature_9',
       'Feature_10', 'Feature_11', 'Feature_12', 'Feature_13', 'Feature_14',
       'Feature_15', 'Feature_16', 'Feature_17', 'Feature_18', 'Feature_19',
       'Feature_20', 'Feature_21'],
      dtype='object')

In [ ]:
thinsection_df.to_csv('Thin_Section_Augmented_GLCM_RGB.csv', index=False)